In [1]:
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout,Activation
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras import backend as K
import numpy as np
from keras.preprocessing import image 

Using TensorFlow backend.


In [2]:
img_width , img_height = 150,150
train_data_dir = 'catordog/training_set'
validation_data_dir = 'catordog/test_set'
nb_train_samples = 1000
nb_validation_samples = 100
epochs = 5
batch_size = 20

if K.image_data_format() == 'channels_first':
    input_shape = (3,img_width,img_height)
else:
    input_shape = (img_width,img_height,3)  

In [3]:
train_datagen = ImageDataGenerator(                         
                rescale=(1.0/255),                  # This is the image augmaentation configuration used for testing
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest',
                horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

In [4]:
train_generator = train_datagen.flow_from_directory(directory=train_data_dir,
                                                    target_size=(img_width,img_height),
                                                    color_mode='rgb',                                                
                                                    class_mode='binary',
                                                    batch_size=batch_size,
                                                    interpolation='nearest')

validation_generator = test_datagen.flow_from_directory(directory=validation_data_dir,
                                                       target_size=(img_width,img_height),
                                                       batch_size=batch_size,
                                                       class_mode='binary')

Found 7992 images belonging to 2 classes.
Found 2022 images belonging to 2 classes.


In [5]:
model = Sequential()
# 1st convolution layer & pooling
model.add(Conv2D(32,(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# 2nd convolution layer & Pooling
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#3rd convlution layer & pooling 
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

In [6]:
model.fit_generator(train_generator,steps_per_epoch=nb_train_samples,epochs=epochs,validation_data=validation_generator,
                   validation_steps=nb_validation_samples)

Epoch 1/5
1000/1000 [==============================] - 347s 347ms/step - loss: 0.6161 - accuracy: 0.6592 - val_loss: 0.5802 - val_accuracy: 0.7475
Epoch 2/5
1000/1000 [==============================] - 352s 352ms/step - loss: 0.5120 - accuracy: 0.7535 - val_loss: 0.4599 - val_accuracy: 0.7719
Epoch 3/5
1000/1000 [==============================] - 423s 423ms/step - loss: 0.4716 - accuracy: 0.7801 - val_loss: 0.3191 - val_accuracy: 0.7815
Epoch 4/5
1000/1000 [==============================] - 432s 432ms/step - loss: 0.4500 - accuracy: 0.7967 - val_loss: 0.3416 - val_accuracy: 0.8118
Epoch 5/5
1000/1000 [==============================] - 436s 436ms/step - loss: 0.4334 - accuracy: 0.8068 - val_loss: 0.7980 - val_accuracy: 0.7836


In [7]:
model.save_weights('first_try.h5')

In [8]:
img_pred = image.load_img('catordog/test_set/cats/cat.4016.jpg',target_size=(150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred,axis=0)

In [9]:
result = model.predict(img_pred)
print(result)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

[[0.]]
cat


Voila!!!!! Model is predicting correctly